In [2]:
import os
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns
from PIL import Image

USE THE GTF FILE TO EXTRACT THE ERV REGIONS

In [10]:
columns = ['seqname', 'source', 'feature', 'start', 'end', 'score','strand','frame', 'attribute']

ensembl_gtf = pd.read_csv('/work/iku/exp1019-cancer-epigenetics-and-ervs/reference/GRCh38-104/ensembl+geve/GRCh38-104.with_geve_v1.gtf', sep = '\t', skiprows = 5, dtype = {0:'str', 7:'str'}, names = columns ) #tego typu 7 nie jestem pewna
print(ensembl_gtf.head())

  seqname          source      feature   start     end score strand frame  \
0       1  ensembl_havana         gene  685679  686673     .      -     .   
1       1  ensembl_havana   transcript  685679  686673     .      -     .   
2       1  ensembl_havana         exon  685679  686673     .      -     .   
3       1  ensembl_havana          CDS  685719  686654     .      -     0   
4       1  ensembl_havana  start_codon  686652  686654     .      -     0   

                                           attribute  
0  gene_id "ENSG00000284662"; gene_version "1"; g...  
1  gene_id "ENSG00000284662"; gene_version "1"; t...  
2  gene_id "ENSG00000284662"; gene_version "1"; t...  
3  gene_id "ENSG00000284662"; gene_version "1"; t...  
4  gene_id "ENSG00000284662"; gene_version "1"; t...  


In [ ]:
FILTER OUT TO ONLY KEEP ROWS FOR ERVs 

In [11]:
def get_gene_id(df):
    df = df.copy()
    pattern = r'gene_id "(.*?)"'
    df['gene_id'] = df['attribute'].str.extract(pattern)
    return df 

In [16]:
ensembl_gtf = get_gene_id(ensembl_gtf)

GET DIFFERENTIALLY EXPRESSED ERVS AND EXTRACT THEIR REGIONS

In [17]:
#read in the overexpressed ERVs that we got form DESEQ2 
file = open('ERV_names.txt')

overexpressed_ERVs = file.read() 
overexpressed_ERVs_list = overexpressed_ERVs.split('\n')[:-1]
erv_df = pd.DataFrame(overexpressed_ERVs_list, columns = ['gene_id'])

In [27]:
#print(erv_df)

KEEP ONLY THE ROWS OF GTF FOR WHICH THE ERV IS PRESENT IN THE OVEREXPRESSED LIST

In [28]:
ensembl_gtf_ERVs = ensembl_gtf[ensembl_gtf['gene_id'].isin(overexpressed_ERVs_list)]
#print(ensembl_gtf_ERVs)


In [29]:
erv_bedgraph_data = ensembl_gtf_ERVs[['seqname','start','end']]
erv_bedgraph_data['is_erv'] = 100
#print(erv_bedgraph_data)

/tmp/ipykernel_716212/492579917.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  erv_bedgraph_data['is_erv'] = 100


In [30]:
output_bedgraph_file = "ervs.bedgraph"
erv_bedgraph_data.to_csv(output_bedgraph_file, sep="\t", header=False, index=False)

In [31]:
#print(erv_df)

READ IN THE COVERAGE DATA

In [7]:
columns = ['chr', 'start', 'end', 'pr_methylated', 'num_methylated', 'num_unmethylated']
s1 = pd.read_csv('/work/iku/exp1019-cancer-epigenetics-and-ervs/01_BS_and_ERV/results/bismark/methylation_call_raw/DNMT1/1/SRR12646267_1_bismark_bt2_pe.bismark.cov.gz', compression = 'gzip', sep = '\t', dtype = {0:'str'}, names = columns)
s2 = pd.read_csv('/work/iku/exp1019-cancer-epigenetics-and-ervs/01_BS_and_ERV/results/bismark/methylation_call_raw/DNMT1/2/SRR12646268_1_bismark_bt2_pe.bismark.cov.gz', compression = 'gzip', sep = '\t',dtype = {0:'str'}, names = columns)
s4= pd.read_csv('/work/iku/exp1019-cancer-epigenetics-and-ervs/01_BS_and_ERV/results/bismark/methylation_call_raw/DNMT1/4/SRR12646270_1_bismark_bt2_pe.bismark.cov.gz', compression = 'gzip', sep = '\t',dtype = {0:'str'}, names = columns)


In [26]:
#print(s1)

In [9]:
def update_dataframe(df):
    df['pr_unmethylated'] = -1*(100 - df['pr_methylated'])
    return df

In [10]:
update_dataframe(s1)
update_dataframe(s2)
update_dataframe(s4)

,chr,start,end,pr_methylated,num_methylated,num_unmethylated,pr_unmethylated
0,7,11095,11095,0.000000,0,1,-100.000000
1,7,11103,11103,0.000000,0,1,-100.000000
2,7,11368,11368,0.000000,0,1,-100.000000
3,7,11385,11385,50.000000,1,1,-50.000000
4,7,11482,11482,33.333333,1,2,-66.666667
...,...,...,...,...,...,...,...
54354877,KI270548.1,510,510,100.000000,1,0,-0.000000
54354878,KI270548.1,690,690,100.000000,1,0,-0.000000
54354879,KI270548.1,713,713,100.000000,1,0,-0.000000
54354880,KI270548.1,719,719,100.000000,1,0,-0.000000


In [16]:
#only keeping the positions that are covered in all three
def merge_by_position(s1,s2,s4):
    merged_df = pd.merge(s1, s2, on=['chr', 'start', 'end'], suffixes=('_df1', '_df2'))
    merged_df = pd.merge(merged_df, s4, on=['chr', 'start', 'end'])
    merged_df['methylation_difference'] = (merged_df['pr_methylated_df1'] + merged_df['pr_methylated_df2'])/2 - merged_df['pr_methylated']
    return merged_df

merged_by_pos = merge_by_position(s1,s2,s4)
#print(merged_by_pos)
    

                 chr  start    end  pr_methylated_df1  num_methylated_df1  \
0                  4  10324  10324          33.333333                   1   
1                  4  10326  10326          33.333333                   1   
2                  4  10352  10352         100.000000                   3   
3                  4  10358  10358         100.000000                   3   
4                  4  10360  10360         100.000000                   3   
...              ...    ...    ...                ...                 ...   
48539956  KI270315.1     90     90          50.000000                   1   
48539957  KI270315.1    125    125         100.000000                   1   
48539958  KI270315.1    175    175         100.000000                   2   
48539959  KI270315.1    275    275         100.000000                   2   
48539960  KI270315.1    284    284         100.000000                   2   

          num_unmethylated_df1  pr_unmethylated_df1  sample_id_df1  \
0    

In [18]:
#only keep the necessary rows
small_df = merged_by_pos[['chr', 'start', 'end', 'methylation_difference']]
print(small_df) 

                 chr  start    end  methylation_difference
0                  4  10324  10324               16.666667
1                  4  10326  10326               16.666667
2                  4  10352  10352              100.000000
3                  4  10358  10358              100.000000
4                  4  10360  10360              100.000000
...              ...    ...    ...                     ...
48539956  KI270315.1     90     90              -50.000000
48539957  KI270315.1    125    125                0.000000
48539958  KI270315.1    175    175               66.666667
48539959  KI270315.1    275    275                0.000000
48539960  KI270315.1    284    284              -16.666667

[48539961 rows x 4 columns]


In [ ]:
output_bedgraph_file = "methylation_difference.bedgraph"
small_df.to_csv(output_bedgraph_file, sep="\t", header=False, index=False)